# Lab

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
random.seed(1) 
np.random.seed(1)

### Load Data

In [2]:
file_path = r"E:\EICU\eicu-collaborative-research-database-2.0\lab.csv.gz"
lab_df = pd.read_csv(file_path, compression="gzip", low_memory=False,)

In [3]:
lab_df.head()

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.0,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.5,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.0,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.6,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.2,7.20,NaN,Units,2155


In [4]:
len(lab_df)

39132531

In [5]:
lab_df = lab_df[lab_df['labresultoffset'] <= 24*60]

In [6]:
len(lab_df)

16077855

In [7]:
lab_df['lab_units'] = lab_df['labmeasurenamesystem'].fillna(lab_df['labmeasurenameinterface'])
lab_df = lab_df.drop(columns=['labmeasurenamesystem', 'labmeasurenameinterface'])

In [8]:
lab_df = lab_df.drop(columns=['labid', 'labresultrevisedoffset', 'labresulttext'])

In [9]:
lab_df.isnull().sum()

patientunitstayid         0
labresultoffset           0
labtypeid                 0
labname                   0
labresult            157204
lab_units            440484
dtype: int64

In [10]:
lab_df.head()

,patientunitstayid,labresultoffset,labtypeid,labname,labresult,lab_units
1,141168,1133,3,PT - INR,2.5,ratio
3,141168,1133,3,PT,26.6,sec
5,141168,231,3,PT - INR,1.7,ratio
8,141168,516,1,BUN,26.0,mg/dL
11,141168,231,3,PT,17.1,sec


In [11]:
lab_df.labname.value_counts()

labname
bedside glucose         906003
potassium               580091
sodium                  547762
Hgb                     517609
glucose                 512297
                         ...  
Procainamide                 7
HSV 1&2 IgG AB titer         7
Amikacin - peak              6
RPR titer                    3
HIV 1&2 AB                   3
Name: count, Length: 158, dtype: int64

In [12]:
lab_df.lab_units.value_counts()

lab_units
mg/dL       3677768
mmol/L      2713230
%           2690497
g/dL        1364887
K/mcL        872075
             ...   
MM3               2
dils              2
IU                1
OD Ratio          1
IV                1
Name: count, Length: 104, dtype: int64

In [13]:
lab_df.labtypeid.value_counts()

labtypeid
1    6754152
3    6102999
7    2023192
4    1149283
2      48196
6         33
Name: count, dtype: int64

In [18]:
selected_labs = ['sodium', 'creatinine','platelets x 1000','bicarbonate', 'chloride','BUN','RBC','potassium']
filtered_lab_df = lab_df[lab_df['labname'].isin(selected_labs)]


In [19]:
filtered_lab_df

,patientunitstayid,labresultoffset,labtypeid,labname,labresult,lab_units
8,141168,516,1,BUN,26.00,mg/dL
20,141168,1133,1,chloride,101.00,mmol/L
29,141168,1133,3,platelets x 1000,213.00,K/mcL
32,141168,516,1,chloride,102.00,mmol/L
38,141168,1133,1,BUN,27.00,mg/dL
...,...,...,...,...,...,...
39132459,3353263,-7,1,bicarbonate,31.00,mmol/L
39132471,3353263,-7,1,BUN,13.00,mg/dL
39132494,3353263,-7,3,RBC,5.44,M/mcL
39132507,3353263,-7,3,platelets x 1000,161.00,K/mcL


In [16]:
filtered_lab_df['labname'].value_counts()

labname
potassium           580091
sodium              547762
chloride            490022
creatinine          484263
BUN                 480490
bicarbonate         455994
platelets x 1000    440008
RBC                 430755
Name: count, dtype: int64

In [17]:
filtered_lab_df['patientunitstayid'].nunique()

189896

In [20]:
# reflect the lab names to the target labs
target_labs = {
    "Serum creatinine min": "creatinine",
    "Serum creatinine max": "creatinine",
    "Sodium max": "sodium",
    "Platelets x1000 min": "platelets x 1000",
    "Platelets x1000 max": "platelets x 1000",
    "Bicarbonate avg": "bicarbonate",
    "Chloride min": "chloride",
    "Chloride avg": "chloride",
    "BUN min": "BUN",
    "RBC max": "RBC",
    "RBC min": "RBC",
    "Potassium max": "potassium",
    "Potassium min": "potassium"
}


lab_summary = filtered_lab_df.groupby(["patientunitstayid", "labname"])["labresult"].agg(['min', 'max', 'mean']).reset_index()





In [21]:
lab_summary

,patientunitstayid,labname,min,max,mean
0,141168,BUN,26.00,27.00,26.500
1,141168,RBC,4.49,4.52,4.505
2,141168,bicarbonate,22.00,26.00,24.000
3,141168,chloride,101.00,102.00,101.500
4,141168,creatinine,1.95,2.30,2.125
...,...,...,...,...,...
1473734,3353263,chloride,101.00,101.00,101.000
1473735,3353263,creatinine,1.06,1.06,1.060
1473736,3353263,platelets x 1000,161.00,164.00,162.500
1473737,3353263,potassium,4.10,4.10,4.100


In [24]:
final_lab_results = lab_summary.pivot(index="patientunitstayid", columns="labname", values=["min", "max", "mean"])
final_lab_results.columns = ['_'.join(col).strip() for col in final_lab_results.columns]
final_lab_results.reset_index(inplace=True)

column_mapping = {
    "min_creatinine": "Serum creatinine min",
    "max_creatinine": "Serum creatinine max",
    "max_sodium": "Sodium max",
    "min_platelets x 1000": "Platelets x1000 min",
    "max_platelets x 1000": "Platelets x1000 max",
    "mean_bicarbonate": "Bicarbonate avg",
    "min_chloride": "Chloride min",
    "mean_chloride": "Chloride avg",
    "min_BUN": "BUN min",
    "max_RBC": "RBC max",
    "min_RBC": "RBC min",
    "max_potassium": "Potassium max",
    "min_potassium": "Potassium min"
}
final_lab_df = final_lab_results.rename(columns=column_mapping)[["patientunitstayid"] + list(column_mapping.values())]

In [25]:
final_lab_df

,patientunitstayid,Serum creatinine min,Serum creatinine max,Sodium max,Platelets x1000 min,Platelets x1000 max,Bicarbonate avg,Chloride min,Chloride avg,BUN min,RBC max,RBC min,Potassium max,Potassium min
0,141168,1.95,2.30,139.0,209.0,213.0,24.0,101.0,101.5,26.0,4.52,4.49,4.2,4.0
1,141178,0.70,0.70,146.0,273.0,273.0,25.0,108.0,108.0,11.0,4.60,4.60,3.6,3.6
2,141179,0.70,0.70,146.0,NaN,NaN,22.0,106.0,107.5,20.0,NaN,NaN,4.2,3.6
3,141194,2.23,2.94,136.0,233.0,298.0,17.5,102.0,105.5,30.0,3.64,3.43,4.6,3.4
4,141196,0.80,0.80,135.0,453.0,453.0,30.0,97.0,97.0,16.0,3.67,3.67,4.1,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189891,3353235,1.04,1.04,137.0,133.0,133.0,24.0,109.0,109.0,16.0,5.49,5.49,4.1,4.1
189892,3353237,1.07,1.07,140.0,128.0,132.0,27.0,101.0,101.0,32.0,4.12,4.08,3.6,3.6
189893,3353251,1.81,2.65,142.0,196.0,233.0,21.2,101.0,103.4,19.0,3.54,3.41,4.4,3.1
189894,3353254,2.21,2.38,141.0,183.0,268.0,22.0,109.0,113.0,42.0,3.89,2.75,5.7,5.0


In [27]:
final_lab_df.to_csv('lab_variable.csv')